# exp001 
https://drive.google.com/file/d/1sV2DSHaIFE-BvPs0POqH3y4qMH883j6N/view?usp=drive_link

In [1]:
import pandas as pd
import tradewhispers as tw

In [2]:
# Load data
stage4 = pd.read_csv("../data/interim/bdm-corpus-2/stage-4.csv")
results_original = tw.map_predictions(pd.read_csv("../results/exp001/results_original.csv"))
results_filtered = tw.map_predictions(pd.read_csv("../results/exp001/results_filtered.csv"))

# Run evaluation
summary_original = tw.evaluate(stage4, results_original, horizons=range(1,21), threshold_pips=25, mode="unfiltered")
summary_filtered = tw.evaluate(stage4, results_filtered, horizons=range(1,21), threshold_pips=1, mode="filtered")

print("\n==== Summary (Unfiltered) ====")
display(summary_original)
print("\n==== Summary (Filtered) ====")
display(summary_filtered)


=== Horizon t+1 (unfiltered) ===
Results rows: 500 | Matched: 500 | Evaluated (has Forward Return t+1): 500 | Traded preds: 364
Confusion Matrix:
           Decrease  Stable  Increase
Decrease        15      19        28
Stable          90      94       176
Increase        19      23        36

Classification Report:
               precision    recall  f1-score   support

    Decrease       0.12      0.24      0.16        62
      Stable       0.69      0.26      0.38       360
    Increase       0.15      0.46      0.23        78

    accuracy                           0.29       500
   macro avg       0.32      0.32      0.26       500
weighted avg       0.54      0.29      0.33       500

Accuracy: 0.29
Cumulative PnL: -0.0024999999999988625

=== Horizon t+2 (unfiltered) ===
Results rows: 500 | Matched: 500 | Evaluated (has Forward Return t+2): 500 | Traded preds: 364
Confusion Matrix:
           Decrease  Stable  Increase
Decrease        21      23        37
Stable          65    

,horizon,n_results,n_matched,n_evaluated,n_traded,accuracy,cum_pnl
0,1,500,500,500,364,0.290000,-0.0025
1,2,500,500,500,364,0.332000,0.0283
2,3,500,500,500,364,0.300000,0.0190
3,4,500,500,500,364,0.336000,0.0805
4,5,500,500,500,364,0.362000,0.1062
5,6,500,500,500,364,0.336000,0.0244
6,7,500,500,498,362,0.305221,-0.0419
7,8,500,500,498,362,0.307229,-0.1071
8,9,500,500,495,359,0.325253,-0.0260
9,10,500,500,495,359,0.323232,-0.0580



==== Summary (Filtered) ====


,horizon,n_results,n_matched,n_evaluated,n_traded,accuracy,cum_pnl
0,1,500,500,500,500,0.456000,0.0344
1,2,500,500,500,500,0.470000,0.0238
2,3,500,500,500,500,0.458000,-0.0964
3,4,500,500,500,500,0.482000,-0.0691
4,5,500,500,500,500,0.494000,-0.0215
5,6,500,500,500,500,0.500000,-0.1044
6,7,500,500,498,498,0.485944,-0.1559
7,8,500,500,498,498,0.477912,0.0677
8,9,500,500,497,497,0.501006,-0.0124
9,10,500,500,497,497,0.498994,0.0628


In [3]:
account_original = tw.simulate_trading_account(stage4, results_original, horizons=range(1,21), threshold_pips=25, mode="unfiltered", starting_balance=1000)
account_filtered = tw.simulate_trading_account(stage4, results_filtered, horizons=range(1,21), threshold_pips=1, mode="filtered", starting_balance=1000)


==== Trading Account Simulation (unfiltered) ====


,horizon,final_balance
0,1,1012.872994
1,2,1204.599815
2,3,1112.355845
3,4,1261.677304
4,5,1339.865842
5,6,1198.466901
6,7,1114.203344
7,8,1026.275824
8,9,1241.869980
9,10,1113.786658



==== Trading Account Simulation (filtered) ====


,horizon,final_balance
0,1,1061.738550
1,2,1117.240091
2,3,753.271118
3,4,771.931153
4,5,862.723244
5,6,807.101937
6,7,805.227167
7,8,1052.004778
8,9,986.945858
9,10,1082.937598
